In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch
import torch.nn as nn

# --- 1. Define Model (Must match training exactly) ---
IMG_SIZE = 64
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(128 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
    def forward(self, x): return self.net(x)

# --- 2. Load Weights ---
device = torch.device("cpu") # Export on CPU
model = CNN().to(device)
# Ensure file path is correct!
model.load_state_dict(torch.load("eye_classifier_finetuned.pth", map_location=device, weights_only=True))
model.eval()

# --- 3. Export to ONNX ---
dummy_input = torch.randn(1, 1, 64, 64) # (Batch, Channel, Height, Width)
torch.onnx.export(
    model, 
    dummy_input, 
    "drowsiness_model.onnx",
    input_names=['input'], 
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)
print("✅ Success! 'drowsiness_model.onnx' created.")

✅ Success! 'drowsiness_model.onnx' created.
